# Loading libraries

In [70]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
data = pd.read_csv('/Users/nincha/Projects/Automation_of_rescruitment_process/data_raw.csv')

/var/folders/yg/3fjz6jqs2d11ktgmq_1w7vfm0000gn/T/ipykernel_6051/3686892759.py:1: DtypeWarning: Columns (4,5,6,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/nincha/Projects/Automation_of_rescruitment_process/data_raw.csv')


In [72]:
col_info = {}

col_info['num_cols'] = [
    'ExpPeriod', 
    'Salary', 
    'Age', 
    'N_places',
    'N_changeWork', 
    'N_Langs', 
    'N_Unis'
    ]

col_info['text_cols'] = [
    'Title', 'SpecCat', 'City', 'LastPlace', 'LastPlace2', 'LastPos', 
    'LastPos2', 'EduLevel', 'LastUni', 'LastUni2', 'lastUniFaculty', 'lastUni2Faculty'
    ]

col_info['desc_cols'] = [
    'Description', 'LastWorkDesc', 'LastWorkDesc2'
]

col_info['cat_cols'] = [
    'IsEnglish', ' Gender', 'WorkType', 'WorkSchedule', 'Top 10 work', 'RussianNative',
    'LevelEnglish', 'Ready_2_move', 'Ready_4_business_trip', 'Python', 'SQL', 'Power BI', 'Top 10 Uni'
    ]

# Descriptive features preprocessing

In [73]:
def stemmer(text):
    stemmer = SnowballStemmer(language='russian')
    tokenizer = nltk.tokenize.WhitespaceTokenizer()

    tokens = [stemmer.stem(w) for w in tokenizer.tokenize(text)]

    return ' '.join(tokens)

In [74]:
def clean_text(text):

    text = text.lower() 

    text = re.sub(r'\n', '', text)

    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    text = text.split()
    useless_words = stopwords.words("english") + stopwords.words("russian") + ['добрый', 'день', 'hi', 'также']

    text_filtered = [word for word in text if not word in useless_words]

    final_string = ' '.join(text_filtered)

    return final_string

In [75]:
for feat in col_info['desc_cols']:
    data[feat] = data[feat].fillna(' ')
    data[feat] = data[feat].apply(clean_text)
    data[feat] = data[feat].apply(stemmer)

In [76]:
data.to_csv('data_clean.csv')

# Text to categorical 

In [77]:
top_cities = ['Москва', 'Санкт-Петербург', 'Екатеринбург', 'Новосибирск', 'Нижний Новгород']

for i, row in data.iterrows():
    if row['City'] in top_cities:
        data.at[i,'top_city'] = 1
    else:
         data.at[i,'top_city'] = 0

In [78]:
set(data['EduLevel'].values)

{'Higher education',
 'Higher education (Doctor of Science)',
 'Higher education (PhD)',
 'Higher education (bachelor)',
 'Higher education (master)',
 'Incomplete higher education',
 'Secondary education',
 'Secondary special education',
 nan,
 'Высшее образование',
 'Высшее образование (Бакалавр)',
 'Высшее образование (Доктор наук)',
 'Высшее образование (Кандидат наук)',
 'Высшее образование (Магистр)',
 'Неоконченное высшее образование',
 'Среднее образование',
 'Среднее специальное образование'}

In [79]:
bach = ['Высшее образование (Бакалавр)', 'Higher education (bachelor)']
master = ['Higher education (master)', 'Высшее образование (Магистр)']
phd = ['Высшее образование (Кандидат наук)', 'Higher education (PhD)']
doctor = ['Higher education (Doctor of Science)', 'Высшее образование (Доктор наук)']

for i, row in data.iterrows():
    if str(row['EduLevel']) in bach:
        data.at[i, 'EduLevelCat'] = 1
    elif str(row['EduLevel']) in master:
        data.at[i, 'EduLevelCat'] = 2
    elif str(row['EduLevel']) in phd:
        data.at[i, 'EduLevelCat'] = 3
    elif str(row['EduLevel']) in doctor:
        data.at[i, 'EduLevelCat'] = 4
    elif str(row['EduLevel']) == 'Высшее образование' or str(row['EduLevel']) == 'Higher education':
        data.at[i, 'EduLevelCat'] = 1
    else:
        data.at[i, 'EduLevelCat'] = 0


In [80]:
col_info['cat_cols'].append('EduLevelCat')
col_info['cat_cols'].append('top_city')

# Structured data preprocessing

In [81]:
for col_name in col_info['cat_cols']:

    num_unique = len(data[col_name].unique()) 

    if num_unique > 2:
        data[col_name] = data[col_name].fillna(0)

    data[col_name] = data[col_name].astype('category')

    data[col_name] = data[col_name].cat.codes.astype('float')

In [82]:
import re

def fix_salary(price):
    return ''.join(c for c in "abc123def456" if c.isdigit())
data['Salary'] = data['Salary'].apply(fix_salary)

In [83]:
from sklearn.preprocessing import QuantileTransformer

for col_name in col_info['num_cols']:
    num_df = data[col_name]

    num_df.replace('', 0, inplace=True)
    num_df.replace('False', 0, inplace=True)

    col_values = num_df.values.reshape(-1, 1)

    numerical_transformer = QuantileTransformer(output_distribution='normal')

    col_values_norm = numerical_transformer.fit_transform(col_values)

    data[col_name] = col_values_norm

In [85]:
data.drop(['Title', 'IsEnglish', 'SpecCat', 'City', 'EduLevel', 'LastUni', 'LastUni2', 
'lastUniFaculty', 'lastUni2Faculty', 'LastPlace', 'LastPlace2', 'LastPos', 'LastPos2'], axis=1)

,ExpPeriod,Salary,Age,Gender,WorkType,WorkSchedule,N_places,LastWorkDesc,LastWorkDesc2,Top 10 work,...,Ready_4_business_trip,Description,Python,SQL,Power BI,N_Unis,Top 10 Uni,URL,top_city,EduLevelCat
0,-0.888309,-5.199338,-1.694923,2.0,1.0,0.0,-0.621099,1 организова процесс разработк управленческ оп...,1 реализова проект автоматизац подач дан марки...,0.0,...,1.0,1 2021 год законч отлич программ двух диплом э...,1.0,1.0,0.0,0.537980,1.0,https://hh.ru/resume/c107054800051cb16b0039ed1...,1.0,1.0
1,-0.404717,-5.199338,-0.613509,2.0,1.0,0.0,0.464007,создан powerbi репорт дашборд средн высш менед...,управлен ликвидн мониторинг прогнозирован движ...,0.0,...,1.0,им экономическ образован продвинут пользовател...,1.0,0.0,0.0,-5.199338,0.0,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,1.0,1.0
2,-2.483413,-5.199338,0.564267,3.0,1.0,0.0,-1.075207,— выполнен задач различн заказчик создан систе...,— реализова отчетн компан помощ интерактивн ви...,0.0,...,1.0,зовут юл агая реализова отчетн power bi различ...,0.0,0.0,0.0,-5.199338,0.0,https://hh.ru/resume/8c6333c200090aa8b60039ed1...,1.0,1.0
3,-0.042668,-5.199338,-0.149853,2.0,1.0,0.0,1.027247,формирован разработк дашборд power bi анализ п...,разработк аналитическ отчет сбор информац отче...,0.0,...,1.0,уверен пользовател желан оптимизац рационализа...,0.0,1.0,1.0,-5.199338,0.0,https://hh.ru/resume/993bd0980002dd6cff0039ed1...,1.0,1.0
4,-1.239352,-5.199338,-1.985876,3.0,1.0,0.0,-0.621099,создан поддержк развит критическ важн отчетн о...,подготовк отчетн построен дашборд tableau даль...,1.0,...,1.0,последн год проход обучен возможн работа gpa 8...,1.0,1.0,1.0,-5.199338,1.0,https://hh.ru/resume/628596ac000657935b0039ed1...,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,0.191865,-5.199338,-0.271787,0.0,1.0,0.0,0.464007,написан бизнестребован согласн поставлен задач...,формирован ежедневн ежемесячн ежеквартальн отч...,0.0,...,0.0,семейн положен женат дет жела график работ 52,0.0,0.0,0.0,-5.199338,0.0,https://hh.ru/resume/501656c1000405dcd70039ed1...,1.0,1.0
50402,0.857097,-5.199338,NaN,1.0,1.0,0.0,-5.199338,анализ сведен,,0.0,...,0.0,коммуникабельн стрессоустойчив работоспособн,0.0,0.0,0.0,-5.199338,0.0,https://hh.ru/resume/8a93d8fc0003dbf3f50039ed1...,1.0,1.0
50403,-0.091712,-5.199338,NaN,1.0,1.0,0.0,0.464007,обязанностиобработк входя звонк сообщен кандид...,мо обязан вход ежедневн обработк обзвон отклик...,0.0,...,1.0,коммуникабельн гибк целеустремлен исполнительн...,0.0,0.0,0.0,-5.199338,0.0,https://hh.ru/resume/dda9e4a900070712290039ed1...,0.0,1.0
50404,1.111702,-5.199338,0.746351,1.0,1.0,0.0,1.027247,• подготовк отчетн мсфо иностра компан юрисдик...,организац работ подготовк управленческ отчетн ...,0.0,...,1.0,зна участк работ универсальн коммерческ банк в...,0.0,0.0,0.0,0.537980,0.0,https://hh.ru/resume/c45896780008000f930039ed1...,1.0,2.0


In [86]:
data.drop(['Title', 'IsEnglish', 'SpecCat', 'City', 'EduLevel', 'LastUni', 'LastUni2', 
'lastUniFaculty', 'lastUni2Faculty', 'LastPlace', 'LastPlace2', 'LastPos', 'LastPos2'], axis=1).to_csv('data_preprocessed.csv', index=False)